# <h1 align="center"><font color="gree">Projeto: Regressão Linear Múltipla</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Este notebook mostrará como criar e consultar uma ``tabela`` ou ``DataFrame`` que você enviou para o ``DBFS``. O [DBFS](https://docs.databricks.com/aws/pt/dbfs) é o ``Sistema de Arquivos do Databricks`` que permite armazenar dados para consulta dentro do Databricks. Este notebook assume que você já possui um arquivo dentro do DBFS que deseja ler.

Este notebook está escrito em ``Python``, então o tipo de célula padrão é Python. No entanto, você pode usar diferentes linguagens utilizando a sintaxe %LANGUAGE. ``Python``, ``Scala``, ``SQL`` e ``R`` são todos suportados.

In [0]:
file_location = "/Volumes/cat_multiple_linear_regression/schema_ml_mult_regression/vol_ml_regression_csv"

# As opções aplicadas são para arquivos CSV. Para outros tipos de arquivos, elas serão ignoradas:
df =spark.read.csv(file_location, header=True, inferSchema=True)
#df.show()
display(df)

Databricks visualization. Run in Databricks to view.

In [0]:
df.printSchema()

In [0]:
df.columns

In [0]:
### Handling Categorical Features
from pyspark.ml.feature import StringIndexer

In [0]:
df.show()

In [0]:
indexer=StringIndexer(inputCol="sex",outputCol="sex_indexed")
df_r=indexer.fit(df).transform(df)
df_r.show()

In [0]:
indexer=StringIndexer(inputCols=["smoker","day","time"],outputCols=["smoker_indexed","day_indexed",
                                                                  "time_index"])
df_r=indexer.fit(df_r).transform(df_r)
df_r.show()

In [0]:
df_r.columns

In [0]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['tip','size','sex_indexed','smoker_indexed','day_indexed',
                          'time_index'],outputCol="Independent Features")
output=featureassembler.transform(df_r)

In [0]:
output.select('Independent Features').show()

In [0]:
output.show()

In [0]:
finalized_data=output.select("Independent Features","total_bill")

In [0]:
finalized_data.show()

In [0]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol='Independent Features', labelCol='total_bill')
regressor=regressor.fit(train_data)

In [0]:
regressor.coefficients

In [0]:
regressor.intercept

In [0]:
### Predictions
pred_results=regressor.evaluate(test_data)

In [0]:
## Final comparison
pred_results.predictions.show()

In [0]:
### PErformance Metrics
pred_results.r2,pred_results.meanAbsoluteError,pred_results.meanSquaredError